In [1]:
#https://www.kaggle.com/stonet2000/lux-ai-season-1-jupyter-notebook-tutorial
from kaggle_environments import make
#env = make("lux_ai_2021", configuration={"seed": 42, "loglevel": 1, "annotations": True}, debug=True)
#steps = env.run(["simple_agent", "simple_agent"])
#env.render(mode="ipython", width=1200, height=800)

#env = make("lux_ai_2021", configuration={"width": 12, "height": 12, "loglevel": 2, "annotations": True}, debug=False)
#steeps = env.run(['agent.py', 'simple_agent'])
#env.render(mode="ipython", width=900, height= 800)
# for kaggle-environments
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys
import random

Loading environment football failed: No module named 'gfootball'


In [2]:
### Define helper functions

def is_in_map(pos):
    x = pos.x
    y = pos.y
    w, h = game_state.map.width, game_state.map.height
    
    return (x<w)&(y<w)&(x>=0)&(y>=0)

# Redefine Position.direction_to to avoid citytiles
def direction_to_avoid_citytiles(unit, game_state, target_pos):
        """
        Return closest position to target_pos from this position, if it is not a citytile (except target_pos)
        """
        DIRECTIONS = Constants.DIRECTIONS

        check_dirs = [
            DIRECTIONS.NORTH,
            DIRECTIONS.EAST,
            DIRECTIONS.SOUTH,
            DIRECTIONS.WEST,
        ]
        #print("====DEBUG==== direction_to_avoid_citytiles")
        #print("====DEBUG==== unit.pos",unit.pos)
        #print("====DEBUG==== target_pos",target_pos)
        
        #closest_dist = unit.pos.distance_to(target_pos)
        
        closest_dir = DIRECTIONS.CENTER
        
        i=0
        while i<4 and (unit.pos.translate(check_dirs[i], 1) != target_pos and game_state.map.get_cell_by_pos(unit.pos.translate(check_dirs[i], 1)).citytile!=None):
            i+=1
        
        
        if i<4:
            direction = check_dirs[i]
            #print("====DEBUG==== direction",direction)
            
            newpos = unit.pos.translate(direction, 1)
            #print("====DEBUG==== newpos",newpos)
            #if newpos == target_pos or game_state.map.get_cell_by_pos(newpos).citytile==None:
            closest_dist = unit.pos.translate(check_dirs[i], 1).distance_to(target_pos)
            #print("====DEBUG==== closest_dist",closest_dist)

            #closest_dir = DIRECTIONS.CENTER
            closest_dir = check_dirs[i]
            #print("====DEBUG==== closest_dir",closest_dir)
            for direction in check_dirs:
                newpos = unit.pos.translate(direction, 1)
                if is_in_map(newpos):
                    
                    #print("====DEBUG==== newpos",newpos)
                    #print("====DEBUG==== game_state.map.get_cell_by_pos(newpos).citytile",game_state.map.get_cell_by_pos(newpos).citytile)
                    if game_state.map.get_cell_by_pos(newpos) != None:
                        if newpos == target_pos or game_state.map.get_cell_by_pos(newpos).citytile==None:
                            #print("====DEBUG==== dans le if")
                            dist = target_pos.distance_to(newpos)
                            #print("====DEBUG==== dist",dist)
                            if dist < closest_dist:
                                closest_dir = direction
                                closest_dist = dist
                                #print("====DEBUG==== MAJ. closest_dir",closest_dir,"closest_dist",closest_dist)

                #print("====DEBUG==== closest_dir",closest_dir)
        
            
        return closest_dir


def get_list_of_carts(player):
    res = []
    for u in player.units:
        if u.is_cart():
            res.append(u)
    return res

# +++++++++++++++++++++++++++++++++++++++++++++++++
#   FIND RESOURCES IN THE MAP
# +++++++++++++++++++++++++++++++++++++++++++++++++
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles


# +++++++++++++++++++++++++++++++++++++++++++++++++
#   FIND RESOURCES CLOSEST TO UNIT FULL OF CARGO
# +++++++++++++++++++++++++++++++++++++++++++++++++
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = math.inf
    for resource_tile in resource_tiles:
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            if (resource_tile.resource.type=='uranium') & (player.researched_uranium()): 
                closest_dist = dist
                closest_resource_tile = resource_tile
            elif (resource_tile.resource.type=='coal') & (player.researched_coal()): 
                closest_dist = dist
                closest_resource_tile = resource_tile
            elif (resource_tile.resource.type=='wood'):
                closest_dist = dist
                closest_resource_tile = resource_tile
    return closest_resource_tile


# +++++++++++++++++++++++++++++++++++++++++++++++++
#   FIND THE CLOSEST CITYTILE ON THE MAP
# +++++++++++++++++++++++++++++++++++++++++++++++++
def find_closest_city_tile(pos, player):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            for city_tile in city.citytiles:
                dist = city_tile.pos.distance_to(pos)
                if dist < closest_dist:
                    closest_dist = dist
                    closest_city_tile = city_tile
    return closest_city_tile

# +++++++++++++++++++++++++++++++++++++++++++++++++
#   FIND THE CLOSEST CITYTILE OF THE GIVEN CITY
# +++++++++++++++++++++++++++++++++++++++++++++++++
def find_closest_city_tile_in_city(pos, player, cit):
    closest_city_tile = None
    if len(player.cities) > 0:
        closest_dist = math.inf
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for k, city in player.cities.items():
            if city == cit:
                for city_tile in city.citytiles:
                    dist = city_tile.pos.distance_to(pos)
                    if dist < closest_dist:
                        closest_dist = dist
                        closest_city_tile = city_tile
    return closest_city_tile


# +++++++++++++++++++++++++++++++++++++++++++++++++
#   FIND THE CLOSEST CELL IN A GIVEN LIST OF CELLS
# +++++++++++++++++++++++++++++++++++++++++++++++++
def find_closest_cell_in_list(pos, list_cells):
    closest_cell = None
    closest_dist = math.inf
    if len(list_cells) > 0:
        
        # the cities are stored as a dictionary mapping city id to the city object, which has a citytiles field that
        # contains the information of all citytiles in that city
        for c in list_cells:
            dist = c.pos.distance_to(pos)
            if dist < closest_dist:
                closest_dist = dist
                closest_cell = c
    return closest_cell, closest_dist


# ===========================================
#  FIND THE TILES ADJACENT TO GIVEN POSITION
# ============================================
def adjacent_tiles(pos):
    ''' Get adjacent tiles given a tile'''
    x1,y1 = pos.x, pos.y
    w, h = game_state.map.width, game_state.map.height
    adjacent_tiles_pos = [(x1-1,y1),(x1+1,y1),(x1,y1+1),(x1,y1-1)]
    return [ game_state.map.get_cell(x,y) for (x,y) in adjacent_tiles_pos if (x<w)&(y<w)&(x>=0)&(y>=0) ]

# ==============================================================
#  FIND THE TILE TO BUILD ADJACENT TO CLOSEST CITYTILE
# ==============================================================
def building_tiles(player, pos):
    list_adjacent_cells_to_city = []
    cities = []
    for k, city in player.cities.items():
        cities.append(city.citytiles)
    
    for city in cities:
        for citytile in city:
            neighborhood = adjacent_tiles(citytile.pos) # list of Cell
            for n in neighborhood:
                if n not in list_adjacent_cells_to_city and (n.has_resource()==False) & (n.citytile==None) :
                    list_adjacent_cells_to_city.append(n)
    closest_cell, closest_dist = find_closest_cell_in_list(pos, list_adjacent_cells_to_city)       
    return closest_cell

    
    '''
    tile_to_build = None
    cell_position = game_state.map.get_cell_by_pos(pos)
    if (cell_position.has_resource() == False):
        tile_to_build = game_state.map.get_cell_by_pos(pos)
    else:
        tiles_next_pos = adjacent_tiles(pos)
        for cell in tiles_next_pos:
            cell_found = False
            if (cell.has_resource()==False) & (cell.citytile==None) & (cell_found==False):
                tile_to_build = cell
                cell_found = True
    return tile_to_build
    '''
def citytiles_sorted_by_fuel_capacity(pos, player):
    """
    Return the list of the closest citytiles belonging to different cities, sorted by amount of fuel available in the corresponding cities
    The number of elements of the resulting list is equal to the number of cities
    """
    res=[]
    if len(player.cities) > 0:
        for k, city in player.cities.items():
            city_tile = find_closest_city_tile_in_city(pos, player, city)
            res.append({"tile": city_tile, "fuel":city.fuel})
            
    if res!=[]:
        res= sorted(res, key=lambda x: x["fuel"], reverse=False)
    return res

# ==============================================================
#  COMPUTE TIME BEFORE NIGHTFALL
# ==============================================================
def time_before_nightfall(observation):
    """
    Returns the number of rounds before nightfall, 0 if it is the night
    """
    step = observation['step']%40
    return max(0,30-step)



# =========================================================================
#  FLOODFILL ALGORITHM TO GET THE CONNECTED CELLS OF A GIVEN CELL (SAME RESOURCE TYPE)
# =========================================================================
def floodfill(pos):
    res = []
    player_cell = game_state.map.get_cell_by_pos(pos)
    explored = [player_cell]
    to_explore = [player_cell]
    resource_type = player_cell.resource
    
    while to_explore != []:
        current_cell = to_explore.pop()
        if current_cell not in res:
            res.append(current_cell)
            neighborhood = adjacent_tiles(current_cell.pos)
            for a in neighborhood:
                if a not in explored:
                    if a.has_resource():
                        if a.resource.type == resource_type.type:
                            to_explore.append(a)
            explored.append(current_cell)
    return res

# =======================================
#  FIND ALL SETS OF RESOURCES OF THE MAP
# =======================================
def get_sets_of_resources(city_cell):
    """
    Returns a dict {closest_cell : [closest_dist, total_ressource]}
    """
    res={}
    list_sets = []
    for x in range(game_state.map.width):
        for y in range(game_state.map.height):
            current_cell = game_state.map.get_cell(x,y)
            if current_cell.has_resource() :
                if current_cell not in (item for sublist in list_sets for item in sublist):
                    list_sets.append(floodfill(current_cell.pos))
                    
    for s in list_sets:
        total_resource = 0
        for c in s:
            total_resource += c.resource.amount
        
        closest_cell, closest_dist = find_closest_cell_in_list(city_cell.pos, s)
        # Added : keep only sets of resources at sufficient distance
        if closest_dist>2:
            res[closest_cell] = [closest_dist,total_resource]
    
    if res!=[]:
        res = dict(sorted(res.items(), key=lambda item: item[1][0]))
    return res

# Strategy for carts : select randomly the adjacent tile which has the less road value
def move_cart(actions, unit, next_direction, positions_occupied_next_round):
    #print("==== DEBUG ==== Move cart. Pos", unit.pos)
    neighborhood = adjacent_tiles(unit.pos)
    random.shuffle(neighborhood)
    
    neighbor = neighborhood[0]
    min_road_value = neighbor.road
    
    for n in neighborhood[1:]:
        if n.citytile == None and n.has_resource()== False and n.road<min_road_value:
            neighbor = n
            min_road_value = n.road
    
    next_direction = unit.pos.direction_to(neighbor.pos)
    position_wanted = neighbor.pos.translate(next_direction, 1)
    
    if is_in_map(position_wanted):

        #print("==== DEBUG ==== next_direction", next_direction)
        #print("==== DEBUG ==== position_wanted", position_wanted)

        # If no unit want to go there, move. Stay here else.
        if position_wanted not in positions_occupied_next_round:
            positions_occupied_next_round.append(position_wanted)
            action = unit.move(next_direction)
            actions.append(action)
        else:
            neighborhood = adjacent_tiles(unit.pos)
            random.shuffle(neighborhood)
            for a in neighborhood:
                if a.pos not in positions_occupied_next_round:
                    positions_occupied_next_round.append(a.pos)
                    action = unit.move(next_direction)
                    actions.append(action)
                    break


In [3]:
game_state = None
def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    
    actions = []

    ### AI Code goes down here! ### 
    
    

    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height
    
    # add debug statements like so!
    if game_state.turn == 0:
        print("Agent is running!", file=sys.stderr)
        
    # DEBUG
    #print("###",time_before_nightfall(observation))
        
    
    
    # Shared data
    
    global positions_occupied_next_round
    global initial_city_cell
    global sets_of_resources
    global info_carts
    
    
    
    # Clear the list positions_occupied_next_round
    positions_occupied_next_round = []
    
    
    # Get the cell of the initial city
    cities = list(player.cities.values())
    if game_state.turn == 0:
        initial_city_cell = game_state.map.get_cell_by_pos(cities[0].citytiles[0].pos)
    
    # Update the list of sets of resources
    sets_of_resources = get_sets_of_resources(initial_city_cell)
    
    # Update list of carts
    list_of_carts = get_list_of_carts(player)
    #if game_state.turn == 0:
    info_carts={}
    for i in list_of_carts:
        info_carts[i] = True # True : the cart must go to its target, False : it must return to the city
    
    

    
    resource_tiles = find_resources(game_state)
    
    # Source : https://www.kaggle.com/thalesgaluchi/lux-ai-first-approach
    # ============
    #    CITIES
    # ============
    tiles_in_cities = [ ct for k,city in player.cities.items() for ct in city.citytiles ]
    for k, city in player.cities.items():
        #         if (len(tiles_in_cities)>(len(player.units))):
        for i,city_tile in enumerate(city.citytiles):
            if city_tile.can_act():
                '''
                if player.city_tile_count>=5 and :
                    action = city_tile.research()
                    print(i,"<<< Research >>>", action)
                    actions.append(action)
                else :
                    if (i%2==0):
                        action = city_tile.build_worker()
                        print(i,"<<< Build Worker >>>", action)
                        actions.append(action)
                    else:
                        action = city_tile.build_cart()
                        print(i,"<<< Build Cart >>>", action)
                        actions.append(action)
                '''
                
                '''
                    Worker   : 5/10
                    Research : 3/10
                    Cart     : 2/10
                '''
                N=10
                if (i%N<5):
                    if len(player.units) < player.city_tile_count:
                        action = city_tile.build_worker()
                        #print("<<< Build Worker >>>", action)
                        actions.append(action)
                elif (5<=i%N<8):
                    action = city_tile.research()
                    #print('<<< Research >>> ')
                    actions.append(action)
                
                else:
                    if len(player.units) < player.city_tile_count:
                        action = city_tile.build_cart()
                        #print("<<< Build Cart >>>", action)
                        actions.append(action)
                
                
    
    # ===============================
    #    UNITS (workers and carts) 
    # ===============================
    for i, unit in enumerate(player.units):
        if unit.is_worker() and unit.can_act():
            if player.city_tile_count==0:
                #print('==DEBUG== EMERGENCY. Worker build a city')
                action = unit.build_city()
                actions.append(action)
            else:
                
                closest_city_tile = find_closest_city_tile(unit.pos, player)

                # FOR DEBUG
                '''
                print("##############################################")
                tmp = citytiles_sorted_by_fuel_capacity(unit.pos, player)
                for i in tmp:
                    print(i["tile"].cityid,i["fuel"])
                print("##############################################")
                '''
                '''
                # If night is comming, try to go back to the closest city
                # According to the rules, number of units <= cities
                if (time_before_nightfall(observation) == unit.pos.distance_to(closest_city_tile.pos)):
                    next_direction = unit.pos.direction_to(closest_city_tile.pos)
                    action = unit.move(next_direction)
                    actions.append(action)

                else:
                '''
                # If there is still space in the cargo, move to the closest resource tile (if possible)
                if unit.get_cargo_space_left()>0:
                    closest_resource_tile = find_closest_resources(unit.pos,player,resource_tiles)
                    next_direction = unit.pos.direction_to(closest_resource_tile.pos)
                    position_wanted = unit.pos.translate(next_direction, 1)
                    if is_in_map(position_wanted):
                        # If no unit want to go there, move. Stay here else.
                        if position_wanted not in positions_occupied_next_round:
                            positions_occupied_next_round.append(position_wanted)
                            action = unit.move(next_direction)
                            actions.append(action)
                        else:
                            neighborhood = adjacent_tiles(unit.pos)
                            random.shuffle(neighborhood)
                            for a in neighborhood:
                                if a.pos not in positions_occupied_next_round:
                                    next_direction = unit.pos.direction_to(a.pos)
                                    positions_occupied_next_round.append(a.pos)
                                    action = unit.move(next_direction)
                                    actions.append(action)
                                    break
                    '''

                    position_wanted = unit.pos.translate(next_direction, 1)
                    # If no unit want to go there, move. Stay here else.
                    if position_wanted not in positions_occupied_next_round:
                        positions_occupied_next_round.append(position_wanted)
                        action = unit.move(next_direction)
                        actions.append(action)
                    '''
                    #tiles_occupied_next_round.

                # If the cargo is full.
                else:

                    city_tiles_fuel = citytiles_sorted_by_fuel_capacity(unit.pos, player)

                    go_to_city_tile = False    
                    for ct in city_tiles_fuel:
                        required_fuel_city = player.cities[ct['tile'].cityid].get_light_upkeep()*10

                        if ct['fuel'] < required_fuel_city:
                            next_direction = unit.pos.direction_to(ct['tile'].pos)

                            position_wanted = unit.pos.translate(next_direction, 1)
                            if is_in_map(position_wanted):
                                # If no unit want to go there, move. Stay here else.
                                if position_wanted not in positions_occupied_next_round:
                                    positions_occupied_next_round.append(position_wanted)
                                    action = unit.move(next_direction)
                                    actions.append(action)
                                else:
                                    neighborhood = adjacent_tiles(unit.pos)
                                    random.shuffle(neighborhood)
                                    for a in neighborhood:
                                        if a.pos not in positions_occupied_next_round:
                                            next_direction = unit.pos.direction_to(a.pos)
                                            positions_occupied_next_round.append(a.pos)
                                            action = unit.move(next_direction)
                                            actions.append(action)
                                            break
                                '''

                                position_wanted = unit.pos.translate(next_direction, 1)
                                # If no unit want to go there, move. Stay here else.
                                if position_wanted not in positions_occupied_next_round:
                                    positions_occupied_next_round.append(position_wanted)
                                    action = unit.move(next_direction)
                                    actions.append(action)
                                '''
                                '''
                                action = unit.move(next_direction)
                                actions.append(action)
                                '''
                                go_to_city_tile = True
                                break


                    '''
                    required_fuel_city = player.cities[closest_city_tile.cityid].get_light_upkeep()*10
                    if player.cities[closest_city_tile.cityid].fuel < required_fuel_city:
                        next_direction = unit.pos.direction_to(closest_city_tile.pos)
                        action = unit.move(next_direction)
                        actions.append(action)
                    '''

                    # BUILD
                    if not go_to_city_tile:  
                        tile_to_build = building_tiles(player, unit.pos)
                        #closest_city_tile = find_closest_city_tile(unit.pos,player)
                        #neighborhood = adjacent_tiles(closest_city_tile.pos)
                        #random.shuffle(neighborhood)

                        #tile_to_build=None#neighborhood[0]
                        '''
                        k=0
                        while k<len(neighborhood) and neighborhood[k].has_resource()==False and neighborhood[k].citytile==None:
                            k+=1
                        if k<len(neighborhood):
                            tile_to_build = neighborhood[k]
                        '''
                        #print('==DEBUG== Worker pos',unit.pos)
                        if tile_to_build != None:
                            #print('==DEBUG== tile_to_build.pos',tile_to_build.pos)

                            if unit.pos.equals(tile_to_build.pos):
                                #print('==DEBUG== Worker on tile_to_build')
                                if unit.can_build(game_state.map):
                                    #print('==DEBUG== Worker can build')
                                    action = unit.build_city()
                                    actions.append(action)
                            else:
                                #print('==DEBUG== Worker go to tile_to_build')
                                #next_direction = unit.pos.direction_to(tile_to_build.pos)
                                next_direction = direction_to_avoid_citytiles(unit, game_state,tile_to_build.pos)

                                position_wanted = unit.pos.translate(next_direction, 1)
                                if is_in_map(position_wanted):
                                    #print('==DEBUG== position_wanted',position_wanted)

                                    # If no unit want to go there, move. Stay here else.
                                    if position_wanted not in positions_occupied_next_round:
                                        #print('==DEBUG== Worker reserved position_wanted')
                                        positions_occupied_next_round.append(position_wanted)
                                        action = unit.move(next_direction)
                                        actions.append(action)
                                    else:
                                        #print('==DEBUG== Worker cannot reserve position_wanted')
                                        neighborhood = adjacent_tiles(unit.pos)
                                        random.shuffle(neighborhood)
                                        for a in neighborhood:
                                            if a.pos not in positions_occupied_next_round:
                                                #next_direction = unit.pos.direction_to(a.pos)
                                                #next_direction = direction_to_avoid_citytiles(unit, game_state,tile_to_build.pos)
                                                next_direction = direction_to_avoid_citytiles(unit, game_state,a.pos)
                                                positions_occupied_next_round.append(a.pos)
                                                action = unit.move(next_direction)
                                                actions.append(action)
                                                break
                                '''
                                position_wanted = unit.pos.translate(next_direction, 1)
                                # If no unit want to go there, move. Stay here else.
                                if position_wanted not in positions_occupied_next_round:
                                    positions_occupied_next_round.append(position_wanted)
                                    action = unit.move(next_direction)
                                    actions.append(action)
                                '''
                                '''
                                action = unit.move(next_direction)
                                actions.append(action)
                                '''
                            
                            
        if unit.is_cart() and unit.can_act():
            move_cart(actions, unit, next_direction, positions_occupied_next_round)
            '''
            
            # Random walker
            neighborhood = adjacent_tiles(unit.pos)
            random.shuffle(neighborhood)
            for i in neighborhood :
                next_direction = unit.pos.direction_to(i.pos)
                
                position_wanted = unit.pos.translate(next_direction, 1)
                # If no unit want to go there, move.
                if position_wanted not in positions_occupied_next_round:
                    positions_occupied_next_round.append(position_wanted)
                    action = unit.move(next_direction)
                    actions.append(action)
                    break
            
            '''
            
            '''
            # Build roads to non reached set of resources
            unit_id = list_of_carts.index(unit)
            targets = list(sets_of_resources.keys())
            if unit_id < len(targets):
                target = None
                if info_carts[unit]:
                    target = targets[unit_id]
                else:
                    target = initial_city_cell
                
                if unit.pos == target.pos:
                    info_carts[unit] = not info_carts[unit]
                
                if info_carts[unit]:
                    target = targets[unit_id]
                else:
                    target = initial_city_cell
                    

                next_direction = unit.pos.direction_to(target.pos)

                position_wanted = unit.pos.translate(next_direction, 1)
                # If no unit want to go there, move. Stay here else.
                if position_wanted not in positions_occupied_next_round:
                    positions_occupied_next_round.append(position_wanted)
                    action = unit.move(next_direction)
                    actions.append(action)

            '''
    

    return actions
    

In [4]:
env = make("lux_ai_2021", configuration={"seed": 420, "loglevel": 2, "annotations": True}, debug=True)
steps = env.run([agent, agent])
env.render(mode="ipython", width=800, height=600)

Agent is running!
Agent is running!
[WARN] (match_GkRqSK7O5qmU) - Agent 0 tried to build unit on tile (13, 29) but unit cap reached. Build more CityTiles!; turn 16; cmd: bw 13 29
[WARN] (match_GkRqSK7O5qmU) - Agent 1 tried to build unit on tile (18, 29) but unit cap reached. Build more CityTiles!; turn 16; cmd: bw 18 29
[WARN] (match_GkRqSK7O5qmU) - turn 23; Unit u_3 collided when trying to move e to (13, 28)
[WARN] (match_GkRqSK7O5qmU) - turn 23; Unit u_4 collided when trying to move w to (18, 28)
[WARN] (match_GkRqSK7O5qmU) - Agent 0 tried to build unit on tile (13, 28) but unit cap reached. Build more CityTiles!; turn 24; cmd: bw 13 28
[WARN] (match_GkRqSK7O5qmU) - Agent 1 tried to build unit on tile (18, 28) but unit cap reached. Build more CityTiles!; turn 24; cmd: bw 18 28
[WARN] (match_GkRqSK7O5qmU) - turn 25; Unit u_3 collided when trying to move w to (12, 28)
[WARN] (match_GkRqSK7O5qmU) - turn 25; Unit u_4 collided when trying to move e to (19, 28)
[WARN] (match_GkRqSK7O5qmU) 